In [2]:

import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
from dotenv import load_dotenv
import bs4
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()
from langchain_community.document_loaders import PyPDFDirectoryLoader
## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']

In [40]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

c:\Users\Ritesh Kapoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ritesh Kapoor\.cache\huggingface\hub\models--BAAI--bge-base-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Ritesh Kapoor\AppData\Local\Programs\Python\Python312\Lib\site-packa

In [4]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


In [42]:
embeddings=hf
url = "https://www.datacamp.com/cheat-sheet/getting-started-with-python-cheat-sheet"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

print(docs)
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250) 
final_documents=text_splitter.split_documents(docs) 
vectors=FAISS.from_documents(final_documents,embeddings) 


[Document(page_content='Just a moment...Enable JavaScript and cookies to continue', metadata={'source': 'https://www.datacamp.com/cheat-sheet/getting-started-with-python-cheat-sheet', 'content_type': 'text/html; charset=UTF-8', 'title': 'Just a moment...', 'language': 'en-US'})]


In [44]:

llm=ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")


prompt=ChatPromptTemplate.from_template(
"""
you are a coding test platform.
I have give you a context of python documentation. 
You need to ask questions only on python language  <context>
{context}
<context>
Questions:{input}

"""
)
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)



In [45]:

response=retrieval_chain.invoke({"input":'create 5 random challenging objective type question on python language generate only 3 options out of which 1 is correct. dont tell correct option     '})

print(response['answer'])

1. What is the output of the following code?

```python
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))

a) 55
b) 64
c) 89

---

2. What is the output of the following code?

```python
class A:
    def __init__(self, value):
        self.value = value

    def __add__(self, other):
        return self.value + other.value

a = A(5)
b = A(10)

print(a + b)

a) 15
b) A(15)
c) 510

---

3. What is the output of the following code?

```python
numbers = [1, 2, 3, 4, 5]

print(list(filter(lambda x: x % 2 == 0, numbers)))

a) [1, 3, 5]
b) [2, 4]
c) [1, 2, 3, 4, 5]

---

4. What is the output of the following code?

```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5))

a) 120
b) 125
c) 150

---

5. What is the output of the following code?

```python
def is_palindrome(word):
    if len(word) <= 1:
        retur

In [46]:
from llama_index.llms.groq import Groq
from dotenv import load_dotenv
import os 
load_dotenv()

## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']
llm = Groq(model="mixtral-8x7b-32768", api_key=groq_api_key)

In [47]:
from llama_index.core.llms import ChatMessage
answer=response
messages = [
    ChatMessage(
        role="system", content="you are an intelligent ai assistant.answer the given 5 questions. if the correct answer is not amongst the options then add correct one  "
    ),
    ChatMessage(role="user", content=f"{answer}"),
]
resp = llm.chat(messages)
print(resp)

assistant: 1. What is the output of the following code?

```python
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))
```

a) 55
b) 64
c) 89

---

2. What is the output of the following code?

```python
class A:
    def __init__(self, value):
        self.value = value

    def __add__(self, other):
        return self.value + other.value

a = A(5)
b = A(10)

print(a + b)
```

a) 15
b) A(15)
c) 510

---

3. What is the output of the following code?

```python
numbers = [1, 2, 3, 4, 5]

print(list(filter(lambda x: x % 2 == 0, numbers)))
```

a) [1, 3, 5]
b) [2, 4]
c) [1, 2, 3, 4, 5]

---

4. What is the output of the following code?

```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5))
```

a) 120
b) 125
c) 150

---

5. What is the output of the following code?

```python
def is_palindrome(word):
    if le